In [ ]:
# -*- coding: utf-8 -*-
from miasm.analysis.sandbox import Sandbox_Linux_x86_64
from miasm.analysis.binary import Container
from miasm.os_dep.linux_stdlib import linobjs
from miasm.core.utils import hexdump
from miasm.core.locationdb import LocationDB
from miasm.analysis.dse import DSEPathConstraint
from miasm.analysis.machine import Machine
from miasm.expression.expression import ExprMem, ExprId, ExprInt, ExprAssign
from future.utils import viewitems
from miasm.jitter.csts import PAGE_READ, PAGE_WRITE
import sys

In [ ]:
parser = Sandbox_Linux_x86_64.parser(description='ELF sandboxer')
options = parser.parse_args(args=[])
options.filename = 'flattening_volatile.bin'
options.strategy = 'code-cov'
options.mimic_env = True

In [ ]:
# Need to adjust the context of dse to the one of concrete
def xxx_printf_symb(dse):

    regs = dse.ir_arch.arch.regs
    ret_addr = ExprInt(dse.jitter.get_stack_arg(0), regs.RIP.size)

    dse.update_state({
        regs.RSP: dse.symb.eval_expr(regs.RSP + ExprInt(8, regs.RSP.size)),
        dse.ir_arch.IRDst: ret_addr,
        regs.RAX: ExprInt(6, regs.RAX.size),
        regs.RIP: ret_addr,
    })

In [ ]:
def code_sentinelle(jitter):
    jitter.run = False
    return False

In [ ]:
loc_db = LocationDB()
sb = Sandbox_Linux_x86_64(loc_db, options.filename, options, globals())

with open(options.filename, 'rb') as fstream:
    cont = Container.from_stream(fstream, loc_db)
    
machine = Machine('x86_64')

ret_addr = 0x000000001337beef
sb.jitter.add_breakpoint(ret_addr, code_sentinelle)
sb.jitter.push_uint64_t(ret_addr)

In [ ]:
# init_run need to be placed before DSE is attacked
sb.jitter.init_run(0x1040)

MEM_ARGV_ADDR  = 0x7ff70000
MEM_ARGV1_ADDR = 0x7ff80000

# argv
sb.jitter.vm.add_memory_page(
        MEM_ARGV_ADDR,
        PAGE_READ | PAGE_WRITE,
        b'\x42\x42\x42\x42\x42\x42\x42\x42' +  
        b'\x00\x00\xf8\x7f\x00\x00\x00\x00', 
        'Binary'
    )

# argv[1]
sb.jitter.vm.add_memory_page(
        MEM_ARGV1_ADDR,
        PAGE_READ | PAGE_WRITE,
        b'\x00\x00\x00\x00\x00\x00\x00\x00',
        'Binary'
    )

sb.jitter.cpu.RDI = 0x2             # argc
sb.jitter.cpu.RSI = MEM_ARGV_ADDR   # argv


In [ ]:
# Convert strategy to the correct value
strategy = {
    'code-cov': DSEPathConstraint.PRODUCE_SOLUTION_CODE_COV,
    'branch-cov': DSEPathConstraint.PRODUCE_SOLUTION_BRANCH_COV,
    'path-cov': DSEPathConstraint.PRODUCE_SOLUTION_PATH_COV,
}[options.strategy]

In [ ]:
# Init a DSE instance with a given strategy
dse = DSEPathConstraint(machine, loc_db, produce_solution=strategy)
dse.attach(sb.jitter)
dse.update_state_from_concrete()
dse.add_lib_handler(sb.libs, globals())

In [ ]:
# Symbolize the argument
regs = sb.jitter.ir_arch.arch.regs

argv1 = [] 
for i in range(8):
    # Create an ExprId for argv[1][x]
    argv1.append(ExprId('Argv[1][%d]'%(i), 8))

    # Add constraints because argv1[x] is a readable ascii
    const = dse.z3_trans.from_expr(argv1[i])
    dse.cur_solver.add(31 < const) 
    dse.cur_solver.add(const < 127)

argv1_addr = []
for i in range(8):
    argv1_addr.append(ExprMem(ExprInt(MEM_ARGV1_ADDR + i, 64), 8))

s = {}
for addr, argv in zip(argv1_addr, argv1):
    s[addr] = argv
    
dse.update_state(s)

In [ ]:
todo = set([(ExprInt(0x42, 8), 
            ExprInt(0x41, 8),
            ExprInt(0x41, 8),
            ExprInt(0x41, 8),
            ExprInt(0x41, 8),
            ExprInt(0x41, 8),
            ExprInt(0x41, 8),
            ExprInt(0x41, 8))])
done = set()

snapshot = dse.take_snapshot()

In [ ]:
while todo:
    arg_value = todo.pop()

    if arg_value in done:
        continue

    done.add(arg_value)

    for i in range(8):
        print('Run with ARG = %s' % (arg_value[i]))

    print('---start---')

    dse.restore_snapshot(snapshot, keep_known_solutions=True)

    for i in range(8):
        sb.jitter.eval_expr(ExprAssign(argv1_addr[i], arg_value[i]))

    sb.jitter.init_run(0x1040)
    sb.jitter.set_trace_log(trace_regs=True, trace_new_blocks=False)
    sb.jitter.continue_run(step=False)

    print('---end---')

    if sb.jitter.cpu.RAX == 0:
        print('FOUND!!!')
        argv1_str = ''.join([chr(x) for x in arg_value])
        print(argv1_str)
        break

    for sol_ident, model in viewitems(dse.new_solutions):

        print('Found a solution to reach: %s' % str(sol_ident))
        print('model', model)

        sol_expr = []
        for i in range(8):
            # Get the argument to use as a Miasm Expr
            try:
                sol_value = model.eval(dse.z3_trans.from_expr(argv1[i])).as_long()
            except AttributeError:
                sol_value = 0

            sol_expr.append(ExprInt(sol_value, argv1[i].size))

            print('\tARG[1][%d] = %s' % (i, sol_expr[i]))

        todo.add((sol_expr[0], 
                    sol_expr[1], 
                    sol_expr[2], 
                    sol_expr[3], 
                    sol_expr[4],
                    sol_expr[5], 
                    sol_expr[6], 
                    sol_expr[7]))